2024-09-17 - Ronen Shekel

In [6]:
import matplotlib
matplotlib.use('TKAgg')
%load_ext autoreload
%autoreload 2
import os 
import datetime
import glob 
import numpy as np
import matplotlib.pyplot as plt
from pianoq.simulations.mplc_sim.mplc_sim import MPLCSim
from pianoq.simulations.mplc_sim.mplc_sim_result import MPLCMasks
from pianoq.lab.mplc.singles_scan import signal_scan, idler_scan, get_signal_scanner, get_idler_scanner
from pianoq.lab.mplc.mask_utils import remove_input_modes, add_phase_input_spots, get_imaging_masks
from pianoq_results.scan_result import ScanResult
from pianoq.simulations.mplc_sim.mplc_sim_result import MPLCSimResult
from pianoq.lab.mplc.mplc_device import MPLCDevice
from pianoq.lab.photon_scan import PhotonScanner
from pianoq.misc.misc import run_in_thread, run_in_thread_simple
from pianoq.misc.mplt import mimshow, mplot
from pianoq.simulations.mplc_sim.create_wfm_masks import create_WFM_diffuser_masks
import time 

dir_path = r'G:\My Drive\Projects\MPLC\results\lab\2024_09_17_first_diffuser'
if not os.path.exists(dir_path):
    os.mkdir(dir_path)

d_no = 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Generate masks WFM

In [2]:
from pianoq.simulations.mplc_sim.create_wfm_masks import create_WFM_diffuser_masks

timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
masks_path =    fr'{dir_path}\{timestamp}_diffuser_d{d_no}d.masks'
mplc_res_path = fr'{dir_path}\{timestamp}_diffuser_d{d_no}d.mplcsim'
mplc_sim = create_WFM_diffuser_masks(N_iterations=15, out_path=masks_path)
mplc_sim.res.saveto(mplc_res_path, smaller=True)

100%|██████████| 15/15 [04:11<00:00, 16.78s/it]


In [28]:
path = glob.glob(rf'{dir_path}\*diffuser_d{d_no}d.mplcsim')[0]
mr = MPLCSimResult()
mr.loadfrom(path)
mimshow((np.abs((mr.forward_fields[-1, :]).sum(axis=0))**2)[mr.active_slice])

(<Figure size 640x480 with 2 Axes>, <Axes: >)

# HW

In [2]:
# MPLC
mplc = MPLCDevice()
mplc.restore_location()
print('Got MPLC')

Got MPLC


In [2]:
# MPLC
mplc = MPLCDevice()
mplc.restore_location()
print('Got MPLC')

# Motors
backlash = 0
wait_after_move = 0.3
from pianoq.lab.mplc.consts import thorlabs_x_serial, thorlabs_y_serial
from pianoq.lab.thorlabs_motor import ThorlabsKcubeDC, ThorlabsKcubeStepper
from pianoq.lab.zaber_motor import ZaberMotors
mxi = ThorlabsKcubeDC(thorlabs_x_serial, backlash=backlash, wait_after_move=wait_after_move)
myi = ThorlabsKcubeStepper(thorlabs_y_serial, backlash=backlash, wait_after_move=wait_after_move)
print('Got Thorlabs motors')

zaber_ms = ZaberMotors(backlash=backlash, wait_after_move=wait_after_move)
mxs = zaber_ms.motors[1]
mys = zaber_ms.motors[0]
print('Got Zaber motors')

# Timetagger
from pianoq.lab.time_tagger import QPTimeTagger
from pianoq.lab.mplc.consts import TIMETAGGER_DELAYS, TIMETAGGER_COIN_WINDOW
tt = QPTimeTagger(integration_time=10, remote=True,
                  single_channel_delays=TIMETAGGER_DELAYS, coin_window=TIMETAGGER_COIN_WINDOW)
print('Got Time tagger')

Got MPLC
Got Thorlabs motors
Got Zaber motors
Got Time tagger


# Single counts large scan

In [10]:
def large_singles_scan():
    integration_time = 1
    tt.set_integration_time(integration_time)
    
    # scan signal
    start_x = 10.5
    end_x = 12.5
    start_y = 8.3
    end_y = 11.3
    
    x_pixels = 20
    y_pixels = 30
    pixel_size_x = 0.1
    pixel_size_y = 0.1
    
    timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
    path = f'{dir_path}\\{timestamp}_singles_signal_d{d_no}d_exp_{integration_time}s.scan'
    scanner_sig = PhotonScanner(integration_time, start_x, start_y, x_pixels, y_pixels, pixel_size_x, pixel_size_y, 
                            is_timetagger=True, coin_window=2e-9, saveto_path=path)
    
    scanner_sig.scan(x_motor=mxs, y_motor=mys, ph=tt)
    
    
    # scan idler
    start_x = 7.8
    end_x = 9.8
    start_y = 0.5
    end_y = 3.5
    
    x_pixels = 20
    y_pixels = 30
    pixel_size_x = 0.1
    pixel_size_y = 0.1
    
    
    timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
    path = f'{dir_path}\\{timestamp}_singles_idler_d{d_no}d_exp_{integration_time}s.scan'
    scanner_idl = PhotonScanner(integration_time, start_x, start_y, x_pixels, y_pixels, pixel_size_x, pixel_size_y, 
                            is_timetagger=True, coin_window=2e-9, saveto_path=path)
    
    scanner_idl.scan(x_motor=mxi, y_motor=myi, ph=tt)

In [ ]:
masks_path = glob.glob(rf'{dir_path}\*diffuser_d{d_no}d*.masks')[0]
msks = MPLCMasks()
msks.loadfrom(masks_path)
masks = msks.real_masks
mplc.load_masks(masks, linear_tilts=True)
large_singles_scan()

# Long two-photon scan

In [ ]:
masks_path = glob.glob(rf'{dir_path}\*diffuser_d{d_no}d*.masks')[0]
msks = MPLCMasks()
msks.loadfrom(masks_path)
masks = msks.real_masks
mplc.load_masks(masks, linear_tilts=True)

# Put idler in OK place 
mxi.move_absolute(8.95)
myi.move_absolute(2.1)

# scan signal
start_x = 11
end_x = 12.1
start_y = 8.7
end_y = 10.3
integration_time = 60
tt.set_integration_time(integration_time)

x_pixels = 22
y_pixels = 32
pixel_size_x = 0.05
pixel_size_y = 0.05


timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
path = f'{dir_path}\\{timestamp}_two_ph_speckles_d{d_no}d_exp_{integration_time}s.scan'
scanner = PhotonScanner(integration_time, start_x, start_y, x_pixels, y_pixels, pixel_size_x, pixel_size_y, 
                        is_timetagger=True, coin_window=2e-9, saveto_path=path)

single1s, single2s, coincidences = scanner.scan(ph=tt , x_motor=mxs, y_motor=mys)

# image spots and play 

In [ ]:
masks = get_imaging_masks()
mplc.load_masks(masks)
large_singles_scan()

Moving to starting position...
starting scan
dur: 0. pix: 0, 0. Singles1: 312. Singles2: 645. Coincidence: 0.00.
dur: 1. pix: 0, 1. Singles1: 290. Singles2: 658. Coincidence: 0.00.
dur: 3. pix: 0, 2. Singles1: 315. Singles2: 623. Coincidence: 0.00.
dur: 5. pix: 0, 3. Singles1: 311. Singles2: 706. Coincidence: 0.00.
dur: 6. pix: 0, 4. Singles1: 300. Singles2: 666. Coincidence: 0.00.
dur: 8. pix: 0, 5. Singles1: 286. Singles2: 788. Coincidence: 0.00.
dur: 9. pix: 0, 6. Singles1: 278. Singles2: 1402. Coincidence: 0.00.
dur: 11. pix: 0, 7. Singles1: 280. Singles2: 1862. Coincidence: 0.00.
dur: 12. pix: 0, 8. Singles1: 284. Singles2: 1266. Coincidence: 0.00.
